<a href="https://colab.research.google.com/github/ShaikJasmin11/Chat-Bot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy pandas nltk transformers torch

In [ ]:
from google.colab import files
uploaded = files.upload()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import pandas as pd
import re

# Load the movie lines
lines_file = "movie_lines.txt"
# Specify the encoding as 'latin-1'
lines = pd.read_csv(lines_file, sep=" \+\+\+\$\+\+\+ ", header=None, engine="python", names=["lineID", "characterID", "movieID", "character", "text"], encoding='latin-1')
# If 'latin-1' doesn't work, try 'cp1252'
# lines = pd.read_csv(lines_file, sep=" \+\+\+\$\+\+\+ ", header=None, engine="python", names=["lineID", "characterID", "movieID", "character", "text"], encoding='cp1252')

# Load the movie conversations
dialogue_file = "movie_conversations.txt"
conversations = pd.read_csv(dialogue_file, sep=" \+\+\+\$\+\+\+ ", header=None, engine="python", names=["characterID1", "characterID2", "movieID", "utteranceIDs"], encoding='latin-1') # Also apply encoding to this line
# If 'latin-1' doesn't work, try 'cp1252'
# conversations = pd.read_csv(dialogue_file, sep=" \+\+\+\$\+\+\+ ", header=None, engine="python", names=["characterID1", "characterID2", "movieID", "utteranceIDs"], encoding='cp1252') # Also apply encoding to this line

# Convert the utteranceIDs to a list of IDs (as they are stored as strings)
conversations["utteranceIDs"] = conversations["utteranceIDs"].apply(lambda x: re.findall(r'L[0-9]+', x))

In [ ]:
# Display first few rows of each file to verify loading
print("Movie Lines Data:")
print(lines.head())

print("\nMovie Conversations Data:")
print(conversations.head())

Movie Lines Data:
  lineID characterID movieID character          text
0  L1045          u0      m0    BIANCA  They do not!
1  L1044          u2      m0   CAMERON   They do to!
2   L985          u0      m0    BIANCA    I hope so.
3   L984          u2      m0   CAMERON     She okay?
4   L925          u0      m0    BIANCA     Let's go.

Movie Conversations Data:
  characterID1 characterID2 movieID              utteranceIDs
0           u0           u2      m0  [L194, L195, L196, L197]
1           u0           u2      m0              [L198, L199]
2           u0           u2      m0  [L200, L201, L202, L203]
3           u0           u2      m0        [L204, L205, L206]
4           u0           u2      m0              [L207, L208]


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import re

# Download NLTK data (only required once)
nltk.download('punkt')

# Load your dataset
dialogue_file = "/content/movie_conversations.txt"
lines_file = "/content/movie_lines.txt"

# Specify the encoding as 'latin-1' when reading the files
conversations = pd.read_csv(dialogue_file, sep=" \+\+\+\$\+\+\+ ", header=None, engine='python', encoding='latin-1')
lines = pd.read_csv(lines_file, sep=" \+\+\+\$\+\+\+ ", header=None, engine='python', encoding='latin-1')  # Apply encoding here

# Display the first few rows to identify the correct dialogue column
print(lines.head())

# Assuming the dialogue text is in column 4 (adjust if necessary)
def preprocess(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower(), flags=re.ASCII)
    return text

# Handle missing values and apply preprocessing
lines[4] = lines[4].fillna('').apply(preprocess)

# Preview the preprocessed data
print(lines[4].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


       0   1   2        3             4
0  L1045  u0  m0   BIANCA  They do not!
1  L1044  u2  m0  CAMERON   They do to!
2   L985  u0  m0   BIANCA    I hope so.
3   L984  u2  m0  CAMERON     She okay?
4   L925  u0  m0   BIANCA     Let's go.
0    they do not
1     they do to
2      i hope so
3       she okay
4        lets go
Name: 4, dtype: object


In [ ]:
import pandas as pd

# Assuming the data was loaded as described earlier
# Verify the structure of the data
print(conversations.head())
print(lines.head())

# Ensure 'lines' DataFrame contains the dialogue column at the expected index
dialogue_column_index = 4  # Adjust if necessary, based on your dataset's structure

# Create a dictionary to map line IDs to dialogue text for faster lookup
lines_dict = {}
for _, row in lines.iterrows():
    line_id = row[0]  # Assuming line ID is in the first column
    dialogue = row[dialogue_column_index]  # Adjust based on the column with dialogue text
    lines_dict[line_id] = dialogue

# Convert conversations from string to list format
try:
    conversations[3] = conversations[3].apply(eval)
except Exception as e:
    print(f"Error in eval conversion: {e}")

# Create pairs of input-response dialogues
pairs = []
for conv in conversations[3]:  # Assuming conversation IDs are in the fourth column
    for i in range(len(conv) - 1):
        input_line = lines_dict.get(conv[i])
        response_line = lines_dict.get(conv[i + 1])
        if input_line and response_line:  # Only add pairs if both lines exist
            pairs.append((input_line, response_line))

# Convert to DataFrame
conversation_df = pd.DataFrame(pairs, columns=['Input', 'Response'])

# Display the resulting DataFrame
print(conversation_df.head())

    0   1   2                                 3
0  u0  u2  m0  ['L194', 'L195', 'L196', 'L197']
1  u0  u2  m0                  ['L198', 'L199']
2  u0  u2  m0  ['L200', 'L201', 'L202', 'L203']
3  u0  u2  m0          ['L204', 'L205', 'L206']
4  u0  u2  m0                  ['L207', 'L208']
       0   1   2        3            4
0  L1045  u0  m0   BIANCA  they do not
1  L1044  u2  m0  CAMERON   they do to
2   L985  u0  m0   BIANCA    i hope so
3   L984  u2  m0  CAMERON     she okay
4   L925  u0  m0   BIANCA      lets go
                                               Input  \
0  can we make this quick  roxanne korrine and an...   
1  well i thought wed start with pronunciation if...   
2  not the hacking and gagging and spitting part ...   
3  youre asking me out  thats so cute whats your ...   
4  no no its my fault  we didnt have a proper int...   

                                            Response  
0  well i thought wed start with pronunciation if...  
1  not the hacking and gagging 

In [ ]:
# Step 1: Import necessary libraries
from transformers import pipeline

# Step 2: Load the zero-shot classification pipeline
# (This may download a model for the first time, so ensure you have an internet connection)
nlu_pipeline = pipeline('zero-shot-classification')

# Step 3: Define possible intents
intents = ["greeting", "goodbye", "thanks", "no answer"]

# Step 4: Create a function to classify intent
def classify_intent(input_text):
    result = nlu_pipeline(input_text, candidate_labels=intents)
    return result['labels'][0]  # Get the top label (intent)

# Example usage:
input_text = "Hello, how are you?"
intent = classify_intent(input_text)
print(f"Detected intent: {intent}")

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Detected intent: greeting


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the GPT-2 model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name, cache_dir='/content/cache')
model = GPT2LMHeadModel.from_pretrained(model_name, cache_dir='/content/cache')

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_response(input_text):
    try:
        # Truncate long inputs
        max_input_length = 512
        tokens = tokenizer.encode(input_text, return_tensors='pt').to(device)
        if tokens.shape[-1] > max_input_length:
            tokens = tokens[:, -max_input_length:]

        # Generate response
        outputs = model.generate(tokens, max_length=100, num_return_sequences=1)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    except Exception as e:
        print("Error during response generation:", e)
        return "I'm sorry, I couldn't process your request."

# Test the function
print(generate_response("Hello, how are you?"))

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Hello, how are you?

I'm a little bit of a nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd


In [ ]:
!pip install Flask

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, GPT2LMHeadModel, GPT2Tokenizer
from sklearn.preprocessing import LabelEncoder

# Step 1: Intent Classification (Using DistilBERT for simplicity)

# Load the DistilBERT model and tokenizer
intent_model_name = 'distilbert-base-uncased'
intent_tokenizer = BertTokenizer.from_pretrained(intent_model_name)
intent_model = BertForSequenceClassification.from_pretrained(intent_model_name, num_labels=3)  # Modify for your intent categories

# Intent labels
intent_labels = ["greeting", "weather", "goodbye"]
label_encoder = LabelEncoder()
label_encoder.fit(intent_labels)

# Intent Classification Function
def classify_intent(user_input):
    inputs = intent_tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        outputs = intent_model(**inputs)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=1).item()
    intent = label_encoder.inverse_transform([predicted_class_id])[0]
    return intent

# Step 2: Response Generation (Using GPT-2)

# Load GPT-2 model and tokenizer
response_model_name = 'gpt2'
response_tokenizer = GPT2Tokenizer.from_pretrained(response_model_name)
response_model = GPT2LMHeadModel.from_pretrained(response_model_name)

# Response Generation Function using GPT-2
def generate_response(user_input):
    # Tokenize and encode the user input
    inputs = response_tokenizer.encode(user_input, return_tensors='pt')

    # Generate a response from GPT-2
    outputs = response_model.generate(inputs, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=1.5)

    # Decode and return the generated response
    response_text = response_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response_text

# Example Usage:
user_input = "Hello, how's the weather today?"
intent = classify_intent(user_input)
response = generate_response(user_input)

print(f"Intent: {intent}")
print(f"Response: {response}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


...

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Intent: goodbye
Response: Hello, how's the weather today?

I'm not sure how much rain I've seen in the last few days, but I'm pretty sure it's been pretty good.
 (I've been to the beach a few times, and I can tell you that it was pretty nice.)
. . .
, I think I'll be back in a couple of days. I don't know if I will be able to get back to work, or if it will take me a


In [ ]:
!pip install transformers torch scikit-learn

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.preprocessing import LabelEncoder

# Load pre-trained DistilBERT model and tokenizer for intent classification
intent_tokenizer = BertTokenizer.from_pretrained("distilbert-base-uncased")
intent_model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

# Dummy labels (you can expand this based on your dataset)
intent_labels = ["greeting", "weather", "goodbye"]

# Label encoder for encoding the labels
label_encoder = LabelEncoder()
label_encoder.fit(intent_labels)

def classify_intent(user_input):
    # Tokenize input
    inputs = intent_tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=64)

    # Get model predictions
    with torch.no_grad():
        outputs = intent_model(**inputs)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=1).item()

    # Convert prediction to label
    intent = label_encoder.inverse_transform([predicted_class_id])[0]
    return intent

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.outpu

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load GPT-2 model and tokenizer
response_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
response_model = GPT2LMHeadModel.from_pretrained("gpt2")

def generate_response(user_input):
    # Tokenize user input
    inputs = response_tokenizer.encode(user_input, return_tensors="pt")

    # Generate a response
    outputs = response_model.generate(inputs, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

    # Decode the generated response
    response = response_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [ ]:
!pip install flask pyngrok

In [ ]:
!pip install waitress

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 4.0 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok # Reinstall the pyngrok package

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
from waitress import serve
from threading import Thread

# Initialize Flask app
app = Flask(__name__)

slack_token = '2ol4OdYXMiKT2qFwgi0kZX6M2x6_4FDUekNbJQMBujNZtJrLM'  # Replace with your actual bot token
slack_channel = 'ChatBot'  # Replace with the channel you want to post to

# Function to send a message to Slack
def send_slack_message(message):
    url = 'https://slack.com/api/chat.postMessage'
    headers = {
        'Authorization': f'Bearer {slack_token}',  # Your bot token
        'Content-Type': 'application/json'
    }
    data = {
        'channel': slack_channel,  # The channel you want to post in
        'text': message
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()  # You can return response to debug if needed

# Dummy function for intent classification
# Replace this with your actual logic for intent classification
def classify_intent(user_input):
    # Simple example, classify intent based on keywords
    if "hello" in user_input.lower():
        return "greeting"
    elif "bye" in user_input.lower():
        return "farewell"
    else:
        return "unknown"

# Dummy function for generating a response
# Replace this with actual logic to generate a meaningful response
def generate_response(intent, user_input):
    if intent == "greeting":
        return "Hello! How can I assist you today?"
    elif intent == "farewell":
        return "Goodbye! Have a great day!"
    else:
        return f"I'm sorry, I don't understand the intent of: {user_input}"

@app.route('/slack', methods=['POST'])
def slack_event():
    data = request.json
    user_input = data.get('text')

    # Slack verification challenge
    if "challenge" in data:
        return jsonify({"challenge": data["challenge"]})

    # Handle messages
    if user_input:
        # Classify the intent based on the user input
        intent = classify_intent(user_input)

        # Generate a response based on the intent
        response = generate_response(intent, user_input)

        return jsonify({"text": response})

    # Handle case where 'text' is not provided
    return jsonify({"error": "No text provided"}), 400

# Function to run Flask in a separate thread
def run_flask():
    serve(app, host="0.0.0.0", port=5003)

# Start Flask in a background thread
thread = Thread(target=run_flask)
thread.start()

# Configure ngrok with your authtoken
# Get your authtoken from your ngrok dashboard: https://dashboard.ngrok.com/auth
conf.get_default().auth_token = "2ol4OdYXMiKT2qFwgi0kZX6M2x6_4FDUekNbJQMBujNZtJrLM"  # Replace YOUR_AUTHTOKEN with your actual token

# Disconnect existing tunnels before starting a new one
for tunnel in ngrok.get_tunnels():
    print(f"Disconnecting existing tunnel: {tunnel.public_url}")
    ngrok.disconnect(tunnel.public_url)

# Start ngrok and print public URL
public_url = ngrok.connect(5003)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://ed5d-35-245-206-219.ngrok-free.app" -> "http://localhost:5003"


In [ ]:
!curl -X POST https://ed5d-35-245-206-219.ngrok-free.app/slack -H "content-type: application/json" -d '{"text": "Hello, world!"}'

{"text":"Hello! How can I assist you today?"}


In [ ]:
!curl -X POST https://ed5d-35-245-206-219.ngrok-free.app/slack -H "content-type: application/json" -d '{"text": "Bye"}'

{"text":"Goodbye! Have a great day!"}


In [ ]:
!curl -X POST https://ed5d-35-245-206-219.ngrok-free.app/slack -H "content-type: application/json" -d '{"text": "Namaste"}'

{"text":"I'm sorry, I don't understand the intent of: Namaste"}
